# DM-Control Benchmark Comparison

Compare **Ours** (scaled architecture: 8Q + 3 enc layers) against baselines:
- **TDMPC2** (original)
- **DreamerV3**
- **EfficientZero V2**
- **SimbaV2**
- **SAC**

Sweep: `62_dmcontrol_benchmark` (20 dm-control tasks, 3 seeds each)

In [1]:
import sys
from pathlib import Path

# Add analysis tools to path
NOTEBOOK_DIR = Path.cwd()
ANALYSIS_ROOT = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == "notebooks" else NOTEBOOK_DIR
if str(ANALYSIS_ROOT) not in sys.path:
    sys.path.insert(0, str(ANALYSIS_ROOT))

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tools import wandb_io, config, paths
from tools.aggregations import runs_history_to_frame

## Configuration

In [2]:
# Sweep config
SWEEP_ID = "o46shean"
ENTITY = "thomasevers9"
PROJECT = "tdmpc2-tdmpc2"

# Plot config
MAX_STEP = 200_000  # Plot up to 200k steps
METRIC_KEY = "eval/episode_reward"

# Task name mapping: sweep task name -> baseline CSV name
# Only include mappings where names differ
TASK_NAME_MAP = {
    "ball_in_cup-catch": "cup-catch",
    "finger-turn_easy": "finger-turn-easy",
    "finger-turn_hard": "finger-turn-hard",
    "cartpole-balance_sparse": "cartpole-balance-sparse",
    "cartpole-swingup_sparse": "cartpole-swingup-sparse",
}

# Baseline roots
BASELINE_ROOTS = {
    "TDMPC2": paths.BASELINE_TDMPC2,
    "DreamerV3": paths.BASELINE_DREAMERV3,
    "SAC": paths.BASELINE_SAC,
    "EZ2": paths.PROJECT_ROOT / "results" / "ez2_parsed",
    "SimbaV2": paths.PROJECT_ROOT / "results" / "simbav2_parsed",
    "BMPC": paths.PROJECT_ROOT / "results" / "BMPC_parsed" / "dmcontrol",
}

# Colors for methods
COLORS = {
    "Ours": "#1f77b4",      # Blue
    "TDMPC2": "#2ca02c",    # Green
    "DreamerV3": "#ff7f0e", # Orange
    "EZ2": "#d62728",       # Red
    "SimbaV2": "#9467bd",   # Purple
    "SAC": "#8c564b",       # Brown
    "BMPC": "#17becf",      # Cyan
}

## Load Our Results from W&B

In [3]:
# Fetch sweep runs from W&B (with caching)
# Set force_refresh=True to pull latest data (may fail due to brotli bug)
runs, manifest, source = wandb_io.fetch_sweep_runs(
    entity=ENTITY,
    project=PROJECT,
    sweep_id=SWEEP_ID,
    history_keys=[METRIC_KEY, "_step"],
    use_cache=True,
    force_refresh=False,  # Use cached data; set True to force re-fetch
)
print(f"Loaded {len(runs)} runs from {source}")
print(f"Fetched at: {manifest.get('fetched_at', 'N/A')}")

Loaded 58 runs from cache
Fetched at: 2025-12-18T08:48:39.669001+00:00


In [4]:
# Convert to DataFrame
ours_df = runs_history_to_frame(
    runs,
    metric_key=METRIC_KEY,
    step_keys=["_step"],
    config_to_columns={"task": "task", "seed": "seed"},
)

# Normalize task names to match baseline naming
ours_df["task"] = ours_df["task"].replace(TASK_NAME_MAP)

# Rename metric column to 'reward' for consistency
ours_df = ours_df.rename(columns={METRIC_KEY: "reward"})

print(f"Our results: {len(ours_df)} rows")
print(f"Tasks: {sorted(ours_df['task'].unique().tolist())}")
print(f"Seeds: {sorted(ours_df['seed'].unique().tolist())}")
print(f"Max step: {ours_df['step'].max()}")
ours_df.head()

Our results: 544 rows
Tasks: ['acrobot-swingup', 'cartpole-balance', 'cartpole-balance-sparse', 'cartpole-swingup', 'cartpole-swingup-sparse', 'cheetah-run', 'finger-spin', 'finger-turn-easy', 'finger-turn-hard', 'hopper-hop', 'hopper-stand', 'pendulum-swingup', 'quadruped-run', 'quadruped-walk', 'reacher-easy', 'reacher-hard', 'walker-run', 'walker-stand', 'walker-walk']
Seeds: [42, 43, 44]
Max step: 100000


,task,seed,run_id,step,reward
0,cartpole-balance-sparse,43,0b9upck6,10000,787.599976
1,cartpole-balance-sparse,43,0b9upck6,20000,1000.000000
2,cartpole-balance-sparse,43,0b9upck6,30000,1000.000000
3,cartpole-balance-sparse,43,0b9upck6,40000,1000.000000
4,cartpole-balance-sparse,43,0b9upck6,50000,1000.000000


## Load Baseline Results

In [5]:
def load_baseline_for_task(task: str, method: str, root: Path) -> pd.DataFrame:
    """Load baseline CSV for a task.
    
    Args:
        task: Normalized task name (e.g., 'cup-catch').
        method: Baseline method name.
        root: Path to baseline CSV directory.
    
    Returns:
        DataFrame with columns [step, reward, seed, task, method], or empty if not found.
    """
    csv_path = root / f"{task}.csv"
    if csv_path.exists():
        df = pd.read_csv(csv_path)
        df["task"] = task
        df["method"] = method
        return df
    return pd.DataFrame()


def load_all_baselines(tasks: list, max_step: int) -> pd.DataFrame:
    """Load all baseline results for given tasks.
    
    Args:
        tasks: List of normalized task names.
        max_step: Maximum step to include.
    
    Returns:
        Combined DataFrame of all baselines.
    """
    frames = []
    missing = {method: [] for method in BASELINE_ROOTS}
    
    for method, root in BASELINE_ROOTS.items():
        if not root.exists():
            print(f"Warning: {method} root not found: {root}")
            continue
            
        for task in tasks:
            df = load_baseline_for_task(task, method, root)
            if df.empty:
                missing[method].append(task)
            else:
                df = df[df["step"] <= max_step]
                frames.append(df)
    
    # Report missing
    for method, tasks_missing in missing.items():
        if tasks_missing:
            print(f"{method}: missing {len(tasks_missing)} tasks: {tasks_missing[:5]}..." 
                  if len(tasks_missing) > 5 else f"{method}: missing {tasks_missing}")
    
    if not frames:
        return pd.DataFrame()
    return pd.concat(frames, ignore_index=True)

In [6]:
# Get unique tasks from our results
our_tasks = sorted(ours_df["task"].unique().tolist())
print(f"Our tasks ({len(our_tasks)}): {our_tasks}")

Our tasks (19): ['acrobot-swingup', 'cartpole-balance', 'cartpole-balance-sparse', 'cartpole-swingup', 'cartpole-swingup-sparse', 'cheetah-run', 'finger-spin', 'finger-turn-easy', 'finger-turn-hard', 'hopper-hop', 'hopper-stand', 'pendulum-swingup', 'quadruped-run', 'quadruped-walk', 'reacher-easy', 'reacher-hard', 'walker-run', 'walker-stand', 'walker-walk']


In [7]:
# Load baselines
baselines_df = load_all_baselines(our_tasks, MAX_STEP)
print(f"\nBaseline results: {len(baselines_df)} rows")

if not baselines_df.empty:
    coverage = baselines_df.groupby('method')['task'].nunique()
    print(f"\nTask coverage per method:")
    for method, count in coverage.items():
        print(f"  {method}: {count}/{len(our_tasks)} tasks")


Baseline results: 2456 rows

Task coverage per method:
  BMPC: 19/19 tasks
  DreamerV3: 19/19 tasks
  EZ2: 19/19 tasks
  SAC: 19/19 tasks
  SimbaV2: 19/19 tasks
  TDMPC2: 19/19 tasks


## Combine Data

In [8]:
# Add method column to our results
ours_df["method"] = "Ours"

# Filter our results to max step
ours_filtered = ours_df[ours_df["step"] <= MAX_STEP].copy()

# Combine all data
all_data = pd.concat([ours_filtered, baselines_df], ignore_index=True)
print(f"Total rows: {len(all_data)}")
print(f"Methods: {sorted(all_data['method'].unique().tolist())}")

Total rows: 3000
Methods: ['BMPC', 'DreamerV3', 'EZ2', 'Ours', 'SAC', 'SimbaV2', 'TDMPC2']


## Plotting Functions

In [9]:
def hex_to_rgba(hex_color: str, alpha: float) -> str:
    """Convert hex color to rgba string."""
    hex_color = hex_color.lstrip('#')
    r, g, b = tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))
    return f"rgba({r},{g},{b},{alpha})"


def compute_stats(df: pd.DataFrame, group_cols: list) -> pd.DataFrame:
    """Compute mean and std across seeds."""
    return df.groupby(group_cols).agg(
        mean_reward=("reward", "mean"),
        std_reward=("reward", "std"),
        n_seeds=("reward", "count")
    ).reset_index()

In [10]:
def plot_task_comparison(
    data: pd.DataFrame,
    task: str,
    colors: dict,
) -> go.Figure:
    """Create a comparison plot for a single task.
    
    Args:
        data: DataFrame with columns [step, reward, method, task].
        task: Task name to plot.
        colors: Dict mapping method names to hex colors.
    
    Returns:
        Plotly Figure, or None if no data for task.
    """
    task_data = data[data["task"] == task]
    if task_data.empty:
        return None
    
    stats = compute_stats(task_data, ["method", "step"])
    fig = go.Figure()
    
    # Plot Ours first, then baselines
    methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
    
    for method in methods:
        method_stats = stats[stats["method"] == method].sort_values("step")
        if method_stats.empty:
            continue
        
        color = colors.get(method, "#666666")
        x = method_stats["step"]
        y_mean = method_stats["mean_reward"]
        y_std = method_stats["std_reward"].fillna(0)
        
        # Shaded std region
        fig.add_trace(go.Scatter(
            x=pd.concat([x, x[::-1]]),
            y=pd.concat([y_mean + y_std, (y_mean - y_std)[::-1]]),
            fill="toself",
            fillcolor=hex_to_rgba(color, 0.2),
            line=dict(color="rgba(0,0,0,0)"),
            showlegend=False,
            hoverinfo="skip",
        ))
        
        # Mean line (solid for Ours, dashed for baselines)
        line_style = dict(color=color, width=2)
        if method != "Ours":
            line_style["dash"] = "dash"
        
        fig.add_trace(go.Scatter(
            x=x,
            y=y_mean,
            mode="lines",
            name=method,
            line=line_style,
            hovertemplate=f"{method}<br>Step: %{{x:,}}<br>Reward: %{{y:.1f}}<extra></extra>",
        ))
    
    fig.update_layout(
        title=dict(text=f"<b>{task}</b>", x=0.5),
        xaxis_title="Environment Steps",
        yaxis_title="Episode Reward",
        legend=dict(x=0.02, y=0.98, bgcolor="rgba(255,255,255,0.8)"),
        hovermode="x unified",
        template="plotly_white",
        width=700,
        height=450,
    )
    
    return fig

## Individual Task Plots

In [11]:
# Generate plots for all tasks
figures = {}
for task in our_tasks:
    fig = plot_task_comparison(all_data, task, COLORS)
    if fig is not None:
        figures[task] = fig

print(f"Generated {len(figures)} task plots")

Generated 19 task plots


In [12]:
# Show first 4 plots
for task in list(figures.keys())[:4]:
    print(f"\n--- {task} ---")
    figures[task].show()


--- acrobot-swingup ---



--- cartpole-balance ---



--- cartpole-balance-sparse ---



--- cartpole-swingup ---


## Grid View (All Tasks)

In [13]:
def create_grid_figure(
    data: pd.DataFrame,
    tasks: list,
    colors: dict,
    cols: int = 4,
) -> go.Figure:
    """Create grid of comparison plots for all tasks.
    
    Args:
        data: Combined DataFrame with all methods.
        tasks: List of task names to plot.
        colors: Dict mapping method names to hex colors.
        cols: Number of columns in grid.
    
    Returns:
        Plotly Figure with subplots.
    """
    n_tasks = len(tasks)
    rows = (n_tasks + cols - 1) // cols
    
    fig = make_subplots(
        rows=rows,
        cols=cols,
        subplot_titles=tasks,
        vertical_spacing=0.06,
        horizontal_spacing=0.04,
    )
    
    legend_added = set()
    
    for idx, task in enumerate(tasks):
        row = idx // cols + 1
        col = idx % cols + 1
        
        task_data = data[data["task"] == task]
        if task_data.empty:
            continue
        
        stats = compute_stats(task_data, ["method", "step"])
        methods = ["Ours"] + [m for m in sorted(stats["method"].unique()) if m != "Ours"]
        
        for method in methods:
            method_stats = stats[stats["method"] == method].sort_values("step")
            if method_stats.empty:
                continue
            
            color = colors.get(method, "#666666")
            x = method_stats["step"]
            y_mean = method_stats["mean_reward"]
            y_std = method_stats["std_reward"].fillna(0)
            
            show_legend = method not in legend_added
            legend_added.add(method)
            
            # Shaded region
            fig.add_trace(
                go.Scatter(
                    x=pd.concat([x, x[::-1]]),
                    y=pd.concat([y_mean + y_std, (y_mean - y_std)[::-1]]),
                    fill="toself",
                    fillcolor=hex_to_rgba(color, 0.15),
                    line=dict(color="rgba(0,0,0,0)"),
                    showlegend=False,
                    hoverinfo="skip",
                    legendgroup=method,
                ),
                row=row, col=col
            )
            
            # Mean line
            line_style = dict(color=color, width=1.5)
            if method != "Ours":
                line_style["dash"] = "dash"
            
            fig.add_trace(
                go.Scatter(
                    x=x,
                    y=y_mean,
                    mode="lines",
                    name=method,
                    line=line_style,
                    showlegend=show_legend,
                    legendgroup=method,
                    hovertemplate=f"{method}: %{{y:.0f}}<extra></extra>",
                ),
                row=row, col=col
            )
    
    fig.update_layout(
        height=220 * rows,
        width=1100,
        title_text="<b>DM-Control Benchmark: Ours vs Baselines</b>",
        title_x=0.5,
        showlegend=True,
        legend=dict(x=1.01, y=1, bgcolor="rgba(255,255,255,0.9)"),
        template="plotly_white",
    )
    
    return fig

In [14]:
grid_fig = create_grid_figure(all_data, our_tasks, COLORS, cols=4)
grid_fig.show()

## Summary Table

In [15]:
def create_summary_table(data: pd.DataFrame, eval_step: int = None) -> pd.DataFrame:
    """Create summary table with final mean/std per method per task.
    
    Args:
        data: Combined DataFrame.
        eval_step: Step at which to evaluate. If None, uses max step per method.
    
    Returns:
        DataFrame with one row per task, columns for each method's mean/std.
    """
    results = []
    methods = sorted(data["method"].unique())
    
    for task in sorted(data["task"].unique()):
        task_data = data[data["task"] == task]
        row = {"task": task}
        
        for method in methods:
            method_data = task_data[task_data["method"] == method]
            if method_data.empty:
                row[f"{method}_mean"] = np.nan
                row[f"{method}_std"] = np.nan
                continue
            
            # Get data at eval step or max available
            if eval_step is not None:
                # Find closest step <= eval_step
                available_steps = method_data["step"].unique()
                valid_steps = available_steps[available_steps <= eval_step]
                if len(valid_steps) == 0:
                    step = available_steps.max()
                else:
                    step = valid_steps.max()
            else:
                step = method_data["step"].max()
            
            final_data = method_data[method_data["step"] == step]
            row[f"{method}_mean"] = final_data["reward"].mean()
            row[f"{method}_std"] = final_data["reward"].std()
        
        results.append(row)
    
    return pd.DataFrame(results)

In [16]:
# Summary at 100k steps (our sweep's max)
summary_100k = create_summary_table(all_data, eval_step=100_000)
print("Summary at 100k steps (or max available):")
summary_100k.round(1)

Summary at 100k steps (or max available):


,task,BMPC_mean,BMPC_std,DreamerV3_mean,DreamerV3_std,EZ2_mean,EZ2_std,Ours_mean,Ours_std,SAC_mean,SAC_std,SimbaV2_mean,SimbaV2_std,TDMPC2_mean,TDMPC2_std
0,acrobot-swingup,203.8,55.2,101.3,22.3,307.6,60.7,279.5,15.4,7.7,11.3,102.7,56.9,179.1,85.3
1,cartpole-balance,943.2,88.7,912.6,63.2,939.6,6.3,997.0,3.8,998.7,0.6,999.8,0.0,997.8,0.7
2,cartpole-balance-sparse,996.5,6.8,970.6,27.6,998.5,2.7,1000.0,0.0,808.3,332.1,696.4,419.9,1000.0,0.0
3,cartpole-swingup,770.1,62.2,676.7,40.8,797.2,NaN,877.1,4.6,835.0,9.5,879.7,2.8,839.4,43.1
4,cartpole-swingup-sparse,269.9,329.9,315.3,191.3,663.0,159.2,793.3,57.6,0.4,0.6,557.7,318.9,0.3,0.6
5,cheetah-run,506.0,81.2,335.7,63.0,659.2,22.5,917.2,0.5,310.1,65.3,601.5,87.7,519.1,83.8
6,finger-spin,943.5,28.8,328.3,58.6,753.5,62.6,988.7,2.7,762.4,16.5,744.9,170.1,947.5,34.2
7,finger-turn-easy,559.3,180.6,228.3,119.7,284.2,NaN,843.6,222.3,386.4,178.1,661.6,145.0,488.0,7.4
8,finger-turn-hard,215.2,149.9,189.8,93.0,239.7,77.0,465.9,130.0,61.7,45.6,533.2,192.1,483.6,97.0
9,hopper-hop,58.5,63.1,9.7,16.7,152.1,106.3,306.0,219.2,0.0,0.0,176.4,111.6,13.6,12.8


In [17]:
def format_summary_pretty(df: pd.DataFrame) -> pd.DataFrame:
    """Format summary with 'mean ± std' columns."""
    methods = sorted(set(col.replace("_mean", "") for col in df.columns if col.endswith("_mean")))
    
    formatted = df[["task"]].copy()
    for method in methods:
        mean_col = f"{method}_mean"
        std_col = f"{method}_std"
        if mean_col in df.columns:
            formatted[method] = df.apply(
                lambda r: f"{r[mean_col]:.0f} ± {r[std_col]:.0f}" 
                if pd.notna(r[mean_col]) else "—",
                axis=1
            )
    return formatted

pretty_summary = format_summary_pretty(summary_100k)
pretty_summary

,task,BMPC,DreamerV3,EZ2,Ours,SAC,SimbaV2,TDMPC2
0,acrobot-swingup,204 ± 55,101 ± 22,308 ± 61,279 ± 15,8 ± 11,103 ± 57,179 ± 85
1,cartpole-balance,943 ± 89,913 ± 63,940 ± 6,997 ± 4,999 ± 1,1000 ± 0,998 ± 1
2,cartpole-balance-sparse,996 ± 7,971 ± 28,998 ± 3,1000 ± 0,808 ± 332,696 ± 420,1000 ± 0
3,cartpole-swingup,770 ± 62,677 ± 41,797 ± nan,877 ± 5,835 ± 10,880 ± 3,839 ± 43
4,cartpole-swingup-sparse,270 ± 330,315 ± 191,663 ± 159,793 ± 58,0 ± 1,558 ± 319,0 ± 1
5,cheetah-run,506 ± 81,336 ± 63,659 ± 23,917 ± 1,310 ± 65,602 ± 88,519 ± 84
6,finger-spin,944 ± 29,328 ± 59,753 ± 63,989 ± 3,762 ± 17,745 ± 170,948 ± 34
7,finger-turn-easy,559 ± 181,228 ± 120,284 ± nan,844 ± 222,386 ± 178,662 ± 145,488 ± 7
8,finger-turn-hard,215 ± 150,190 ± 93,240 ± 77,466 ± 130,62 ± 46,533 ± 192,484 ± 97
9,hopper-hop,58 ± 63,10 ± 17,152 ± 106,306 ± 219,0 ± 0,176 ± 112,14 ± 13


## Save Results

In [18]:
# Create output directory
output_dir = paths.notebook_results_dir("08_dmcontrol_benchmark")
print(f"Output directory: {output_dir}")

# Save summary tables
summary_100k.to_csv(output_dir / "summary_100k.csv", index=False)
pretty_summary.to_csv(output_dir / "summary_100k_pretty.csv", index=False)

# Save grid figure
grid_fig.write_html(output_dir / "grid_comparison.html")

# Save individual task figures
for task, fig in figures.items():
    safe_name = task.replace("-", "_")
    fig.write_html(output_dir / f"{safe_name}.html")

print(f"\nSaved:")
print(f"  - summary_100k.csv")
print(f"  - summary_100k_pretty.csv")
print(f"  - grid_comparison.html")
print(f"  - {len(figures)} individual task plots")

Output directory: /gpfs/work4/0/prjs0951/Thomas/Thesis/RL_weather/tdmpc2-with-return-based-auxiliary-tasks/analysis/results/08_dmcontrol_benchmark

Saved:
  - summary_100k.csv
  - summary_100k_pretty.csv
  - grid_comparison.html
  - 19 individual task plots
